In [2]:
#Importing Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
movies=pd.read_csv("/content/drive/My Drive/NLP/Collaborative Filtering/dataset/movies.csv")
ratings=pd.read_csv("/content/drive/My Drive/NLP/Collaborative Filtering/dataset/ratings.csv")

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [7]:
movies.describe()

,movieId
count,9742.000000
mean,42200.353623
std,52160.494854
min,1.000000
25%,3248.250000
50%,7300.000000
75%,76232.000000
max,193609.000000


In [8]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [10]:
ratings.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [11]:
#Merge Movies+Ratings
merged_df=pd.merge(movies,ratings,on='movieId')

In [12]:
merged_df.head(100)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483
...,...,...,...,...,...,...
95,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,269,5.0,850865423
96,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,270,5.0,853918728
97,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,273,5.0,835861234
98,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,274,4.0,1171410158


In [13]:
#dropping unnecessary columns
data=merged_df.drop(["genres","timestamp"],axis=1)

In [14]:
data.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   movieId  100836 non-null  int64  
 1   title    100836 non-null  object 
 2   userId   100836 non-null  int64  
 3   rating   100836 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 3.8+ MB


In [16]:
data.describe()

,movieId,userId,rating
count,100836.000000,100836.000000,100836.000000
mean,19435.295718,326.127564,3.501557
std,35530.987199,182.618491,1.042529
min,1.000000,1.000000,0.500000
25%,1199.000000,177.000000,3.000000
50%,2991.000000,325.000000,3.500000
75%,8122.000000,477.000000,4.000000
max,193609.000000,610.000000,5.000000


In [17]:
#check for null values & duplicates
print("Null values: ",data.isnull().sum().sum())
print("Duplicates: ",data.duplicated().sum())

Null values:  0
Duplicates:  0


In [18]:
#creat user item matrix
pivot_df=data.pivot_table(index='userId',values="rating",columns='title')
pivot_df.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#droping movies that has few user's rating to avoid noise and filling the NaNs with 0 for strandrize the data 
THRES=5
new_df=pivot_df.dropna(thresh=THRES,axis=1).fillna(0)

In [20]:
new_df.head()

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),13 Hours (2016),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),16 Blocks (2006),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",20 Dates (1998),"20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),2046 (2004),21 (2008),21 Grams (2003),21 Jump Street (2012),22 Jump Street (2014),24 Hour Party People (2002),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),3 Ninjas (1992),...,Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men Origins: Wolverine (2009),X-Men: Apocalypse (2016),X-Men: Days of Future Past (2014),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),"Yards, The (2000)",Year One (2009),"Year of Living Dangerously, The (1982)",Yellow Submarine (1968),Yes Man (2008),Yi Yi (2000),Yojimbo (1961),You Can Count on Me (2000),You Don't Mess with the Zohan (2008),You Only Live Twice (1967),You've Got Mail (1998),"You, Me and Dupree (2006)",Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Highness (2011),Youth in Revolt (2009),Zack and Miri Make a Porno (2008),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),[REC] (2007),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
#creat similarity matrix using correlation method
similarty_matrix=new_df.corr(method="pearson")

In [22]:
similarty_matrix

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),13 Hours (2016),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),16 Blocks (2006),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",20 Dates (1998),"20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),2046 (2004),21 (2008),21 Grams (2003),21 Jump Street (2012),22 Jump Street (2014),24 Hour Party People (2002),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),3 Ninjas (1992),...,Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men Origins: Wolverine (2009),X-Men: Apocalypse (2016),X-Men: Days of Future Past (2014),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),"Yards, The (2000)",Year One (2009),"Year of Living Dangerously, The (1982)",Yellow Submarine (1968),Yes Man (2008),Yi Yi (2000),Yojimbo (1961),You Can Count on Me (2000),You Don't Mess with the Zohan (2008),You Only Live Twice (1967),You've Got Mail (1998),"You, Me and Dupree (2006)",Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Highness (2011),Youth in Revolt (2009),Zack and Miri Make a Porno (2008),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),[REC] (2007),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,0.063117,0.235908,-0.023768,0.143482,0.011998,0.087931,0.224052,-0.018608,0.034223,0.009277,0.008331,0.049700,-0.013602,0.176071,0.131993,0.104500,0.064842,0.047271,0.184452,0.092824,0.084066,0.110264,0.166599,0.231762,0.205977,0.187019,0.006377,-0.016983,-0.027152,0.035609,-0.014293,-0.027713,0.097485,0.099271,-0.028021,0.107001,0.117119,-0.028967,0.129193,...,0.075004,0.213522,0.195512,0.087015,0.007059,-0.011004,0.049537,0.119975,0.098020,0.094483,-0.015967,0.068360,0.115369,0.016625,0.069942,0.042313,0.092232,-0.023309,0.021379,0.172502,0.075100,0.201374,0.248535,0.088114,0.257195,-0.018133,-0.013052,0.017477,0.106434,0.032470,0.134701,0.153158,0.101301,0.049897,0.003233,-0.017905,0.187953,0.062174,-0.014025,0.353194
(500) Days of Summer (2009),0.063117,1.000000,0.133949,0.142471,0.273989,0.193960,0.148903,0.142141,0.066567,0.159756,0.135486,0.200135,0.297152,0.090672,0.072835,0.226574,0.087240,0.185825,0.342948,0.038255,0.040104,0.241906,-0.025729,0.001739,-0.025396,0.113616,0.030603,0.274272,0.026493,0.285043,0.094029,0.284552,0.194200,0.128705,0.086447,0.205511,0.048670,0.149167,0.157145,0.039275,...,0.023020,0.031399,0.184228,0.320018,0.238263,0.288610,0.349784,0.274941,0.190061,0.086998,0.134155,-0.029557,0.034780,0.407336,0.041385,0.029789,0.049562,0.144834,-0.012948,0.140981,0.168706,0.066077,0.073476,0.053263,0.053021,0.113952,0.288299,0.374515,0.051286,0.178655,0.068407,0.414585,0.355723,0.252226,0.216007,0.126147,0.053614,0.241092,0.139511,0.125905
*batteries not included (1987),0.235908,0.133949,1.000000,0.035596,0.061144,-0.017106,0.073459,0.106100,-0.012561,0.026377,0.065515,0.065589,0.109313,0.119955,0.034796,0.139575,0.110104,-0.012900,-0.013629,-0.015966,0.081454,-0.018259,-0.010529,0.041545,0.076652,0.152924,0.163847,0.014977,-0.011464,0.053685,-0.021417,0.086968,0.052170,0.266505,0.111092,-0.018915,0.012973,0.104550,-0.019553,0.308344,...,0.090046,0.118265,0.114452,0.189449,0.101980,0.074927,0.052994,0.112370,0.089039,0.062192,-0.010778,-0.012096,0.075837,0.09545

In [25]:
similarty_matrix[:100]

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),13 Hours (2016),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),16 Blocks (2006),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",20 Dates (1998),"20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),2046 (2004),21 (2008),21 Grams (2003),21 Jump Street (2012),22 Jump Street (2014),24 Hour Party People (2002),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),3 Ninjas (1992),...,Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men Origins: Wolverine (2009),X-Men: Apocalypse (2016),X-Men: Days of Future Past (2014),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),"Yards, The (2000)",Year One (2009),"Year of Living Dangerously, The (1982)",Yellow Submarine (1968),Yes Man (2008),Yi Yi (2000),Yojimbo (1961),You Can Count on Me (2000),You Don't Mess with the Zohan (2008),You Only Live Twice (1967),You've Got Mail (1998),"You, Me and Dupree (2006)",Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Your Highness (2011),Youth in Revolt (2009),Zack and Miri Make a Porno (2008),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),[REC] (2007),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,0.063117,0.235908,-0.023768,0.143482,0.011998,0.087931,0.224052,-0.018608,0.034223,0.009277,0.008331,0.049700,-0.013602,0.176071,0.131993,0.104500,0.064842,0.047271,0.184452,0.092824,0.084066,0.110264,0.166599,0.231762,0.205977,0.187019,0.006377,-0.016983,-0.027152,0.035609,-0.014293,-0.027713,0.097485,0.099271,-0.028021,0.107001,0.117119,-0.028967,0.129193,...,0.075004,0.213522,0.195512,0.087015,0.007059,-0.011004,0.049537,0.119975,0.098020,0.094483,-0.015967,0.068360,0.115369,0.016625,0.069942,0.042313,0.092232,-0.023309,0.021379,0.172502,0.075100,0.201374,0.248535,0.088114,0.257195,-0.018133,-0.013052,0.017477,0.106434,0.032470,0.134701,0.153158,0.101301,0.049897,0.003233,-0.017905,0.187953,0.062174,-0.014025,0.353194
(500) Days of Summer (2009),0.063117,1.000000,0.133949,0.142471,0.273989,0.193960,0.148903,0.142141,0.066567,0.159756,0.135486,0.200135,0.297152,0.090672,0.072835,0.226574,0.087240,0.185825,0.342948,0.038255,0.040104,0.241906,-0.025729,0.001739,-0.025396,0.113616,0.030603,0.274272,0.026493,0.285043,0.094029,0.284552,0.194200,0.128705,0.086447,0.205511,0.048670,0.149167,0.157145,0.039275,...,0.023020,0.031399,0.184228,0.320018,0.238263,0.288610,0.349784,0.274941,0.190061,0.086998,0.134155,-0.029557,0.034780,0.407336,0.041385,0.029789,0.049562,0.144834,-0.012948,0.140981,0.168706,0.066077,0.073476,0.053263,0.053021,0.113952,0.288299,0.374515,0.051286,0.178655,0.068407,0.414585,0.355723,0.252226,0.216007,0.126147,0.053614,0.241092,0.139511,0.125905
*batteries not included (1987),0.235908,0.133949,1.000000,0.035596,0.061144,-0.017106,0.073459,0.106100,-0.012561,0.026377,0.065515,0.065589,0.109313,0.119955,0.034796,0.139575,0.110104,-0.012900,-0.013629,-0.015966,0.081454,-0.018259,-0.010529,0.041545,0.076652,0.152924,0.163847,0.014977,-0.011464,0.053685,-0.021417,0.086968,0.052170,0.266505,0.111092,-0.018915,0.012973,0.104550,-0.019553,0.308344,...,0.090046,0.118265,0.114452,0.189449,0.101980,0.074927,0.052994,0.112370,0.089039,0.062192,-0.010778,-0.012096,0.075837,0.09545

In [158]:
#similar movies function
def Recommend(movie,rating):
  recommender=similarty_matrix[movie]*(rating-2.5)#in order to get recommendation right if the rating is low
  recommender=recommender.sort_values(ascending=False)
  return recommender[:10]

In [160]:
Recommend("Heat (1995)",5)

title
Heat (1995)                                                       2.500000
Rock, The (1996)                                                  1.057896
Casino (1995)                                                     1.021208
Rumble in the Bronx (Hont faan kui) (1995)                        0.969080
Léon: The Professional (a.k.a. The Professional) (Léon) (1994)    0.963770
Desperado (1995)                                                  0.961390
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)                         0.961036
Broken Arrow (1996)                                               0.940127
Platoon (1986)                                                    0.892726
Leaving Las Vegas (1995)                                          0.882522
Name: Heat (1995), dtype: float64

In [204]:
movies.head(50)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [200]:
#GETTING MOVIES RECOMMENDATION

#let's say we have a lot of movies all put together and we ask the user to add ratings to get recommendation,so we need to assign movie list with all the 
#the movies available with no respect to the genres, according to the user rating the system will act
def get_recommendation(movie_list):
  similar_movies=pd.DataFrame()
  for m,r in movie_list:
    similar_movies=similar_movies.append(Recommend(m,r),ignore_index=True)
  similar_movies.head()
  print(similar_movies.sum().sort_values(ascending=False)[:20])

In [210]:
#action movies recommendations
action_list=[("Braveheart (1995)",4),
            ("Rumble in the Bronx (Hont faan kui) (1995)",3.5),
            ("Grumpier Old Men (1995)",2),
            ("Toy Story (1995)",1),
            ("Usual Suspects, The (1995)",4),
             ("Clueless (1995)",1),
             ("Mortal Kombat (1995)",4.5)]
get_recommendation(movie_list=action_list)     

Mortal Kombat (1995)                          2.000000
Braveheart (1995)                             1.500000
Usual Suspects, The (1995)                    1.500000
Rumble in the Bronx (Hont faan kui) (1995)    1.000000
Judge Dredd (1995)                            0.971665
Super Mario Bros. (1993)                      0.908903
Crow, The (1994)                              0.876135
Demolition Man (1993)                         0.820855
One, The (2001)                               0.777411
GoldenEye (1995)                              0.766909
Jurassic Park (1993)                          0.714002
Fugitive, The (1993)                          0.706408
Terminator 2: Judgment Day (1991)             0.705725
Pulp Fiction (1994)                           0.705511
Reservoir Dogs (1992)                         0.693265
True Lies (1994)                              0.686812
Batman Forever (1995)                         0.681620
Gremlins (1984)                               0.679415
Snakes on 

In [212]:
#now trying the same movies but with diffrent rattings(not getting action movies recommendations)
comedy_list=[("Braveheart (1995)",2),
            ("Rumble in the Bronx (Hont faan kui) (1995)",1),
            ("Grumpier Old Men (1995)",4),
            ("Toy Story (1995)",4.5),
            ("Usual Suspects, The (1995)",2),
             ("Clueless (1995)",4),
             ("Mortal Kombat (1995)",1)]
get_recommendation(movie_list=comedy_list)     

Toy Story (1995)                              2.000000
Clueless (1995)                               1.500000
Grumpier Old Men (1995)                       1.500000
Nutty Professor, The (1996)                   1.283347
Toy Story 2 (1999)                            0.923522
Groundhog Day (1993)                          0.723079
Independence Day (a.k.a. ID4) (1996)          0.716947
Willy Wonka & the Chocolate Factory (1971)    0.714627
Mission: Impossible (1996)                    0.705694
Bug's Life, A (1998)                          0.690862
Lion King, The (1994)                         0.688497
Babe (1995)                                   0.682272
Grumpy Old Men (1993)                         0.623170
Sleepless in Seattle (1993)                   0.618082
Striptease (1996)                             0.612223
Son in Law (1993)                             0.567903
Bio-Dome (1996)                               0.557907
Father of the Bride Part II (1995)            0.555256
Twister (1